# 1. Data cleaning

## Problem statement

This project is meant as an exercise in basic classification as a training after reading the first three chapters of Hands-On Machine Learning with Scikit-Learn, Keras and Tensorflow by Géron.

### Learning goal: practise implementing a simple end-to-end classification task
Rather than try and achieve very performant results (e.g. optimising an accuracy metric) a focus is to cover all major steps of a typical classification task:
1. Data collection
2. Data cleaning and preprocessing
3. Exploratory data analysis
4. Training of Machine Learning algorithms
5. Optimising the chosen ML technique
6. Drawing insights and conclusions
7. Deployment

The sequential steps are described in separate notebooks.

### Business goal: detect customers that are likely to cancel their credit-card subscription
I found the dataset on Kaggle:
https://www.kaggle.com/sakshigoyal7/credit-card-customers

It comes from a real-life scenario: a bank manager is unhappy that quite a few customers have been cancelling their credit card membership. He/She wants to find out if it's possible to detect such discontent customers early on and win them over by proposing attractive offers before it's too late.
The data has been collected by a Kaggle user, however, it requires some preparation due to:
- text rather than numeric values
- missing values

## Load utilities and programming set up

Let's start by loading the useful libraries, specifying the input data file, and preparing a utility function that will be used to save any figures later on.

In [1]:
# Usual imports
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
assert sklearn.__version__ >= "0.20"

# Pretty plotting
%matplotlib inline
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Data file name (easy to change for a similar project)
DATA_FILE = "BankChurners.csv"

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)  # It's fine if the directory already exists (No action needed)

def save_fig(fig_name, tight_layout=True, fig_extension="png", resolution=300):
    "Save figure ensurring correct directory, extension, resolution and layout"
    path = os.path.join(IMAGES_PATH, fig_name + "." + fig_extension)
    print("Saving figure", fig_name)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Load and have a glimpse at the data

Below I define and call a utility function that loads the data into a pandas DataFrame.
Then, we can have a glimpse at the data.

In [2]:
def load_data(data_path = PROJECT_ROOT_DIR, data_file = DATA_FILE):
    "Load csv data into a pandas DataFrame"
    csv_path = os.path.join(data_path, data_file)
    return pd.read_csv(csv_path)
df = load_data()
df.head(100)
df["Contacts_Count_12_mon"]

0        3
1        2
2        0
3        1
4        0
        ..
10122    3
10123    3
10124    4
10125    3
10126    4
Name: Contacts_Count_12_mon, Length: 10127, dtype: int64

Below we see that most of the columns are numerical (`int64` or `float64`), while a few are text-based (`object`).
Let's inspect possible values of the latter ones.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 23 columns):
 #   Column                                                                                                                              Non-Null Count  Dtype  
---  ------                                                                                                                              --------------  -----  
 0   CLIENTNUM                                                                                                                           10127 non-null  int64  
 1   Attrition_Flag                                                                                                                      10127 non-null  object 
 2   Customer_Age                                                                                                                        10127 non-null  int64  
 3   Gender                                                                           

In [4]:
df.select_dtypes(np.object).head()
text_cols = df.select_dtypes(np.object).columns
text_cols
for col in text_cols:
    print(30*'=')
    print(col)
    print(30*'=')
    print(df[col].value_counts())

Attrition_Flag
Existing Customer    8500
Attrited Customer    1627
Name: Attrition_Flag, dtype: int64
Gender
F    5358
M    4769
Name: Gender, dtype: int64
Education_Level
Graduate         3128
High School      2013
Unknown          1519
Uneducated       1487
College          1013
Post-Graduate     516
Doctorate         451
Name: Education_Level, dtype: int64
Marital_Status
Married     4687
Single      3943
Unknown      749
Divorced     748
Name: Marital_Status, dtype: int64
Income_Category
Less than $40K    3561
$40K - $60K       1790
$80K - $120K      1535
$60K - $80K       1402
Unknown           1112
$120K +            727
Name: Income_Category, dtype: int64
Card_Category
Blue        9436
Silver       555
Gold         116
Platinum      20
Name: Card_Category, dtype: int64




We spot that there are 'Unknown' values in the columns: 'Education_Level', 'Marital_Status' and 'Income_Category'.
They are actually NaN's and let's mark them as such explicitely with `df.replace(.)` and `np.nan`.

In [5]:
df = df.replace("Unknown", np.nan)
print(df["Marital_Status"].value_counts())
df["Income_Category"].value_counts()

# df.isnull().sum() > 0
df.isnull().sum()

Married     4687
Single      3943
Divorced     748
Name: Marital_Status, dtype: int64


CLIENTNUM                                                                                                                                0
Attrition_Flag                                                                                                                           0
Customer_Age                                                                                                                             0
Gender                                                                                                                                   0
Dependent_count                                                                                                                          0
Education_Level                                                                                                                       1519
Marital_Status                                                                                                                         749
Income_Category            

In [6]:
# Show percentage of attrited and existing customers
df["Attrition_Flag"].value_counts(normalize=True) * 100

Existing Customer    83.934038
Attrited Customer    16.065962
Name: Attrition_Flag, dtype: float64

We notice that:
- The first column is a unique client number; we won't need it
- The second one is the client category: either "Existing Customer" or "Attrited Customer". We'll use it as a label for the classification algorithms. Note the imbalance between the existing (84%) and attrited (16%) customers. Hence the problem we're dealing with is actually **imbalanced classification** (more on it below)
- The next columns are various characteristics of each customer, potentially feautures of an ML algorithm
- Some columns are text-based. They need to be translated into numeric values to be ready for an ML technique.

### Imbalanced classification
Having the class ratio significantly different from 1:1 (as it is the case here) may cause an ML algorithm to ignore the minority class. This is obviously undesirable.
While there's no magic solution to the problem, there exist a number of strategies to try out:
1. Use a more suitable performance metric than accuracy, e.g. confusion matrix
2. Over(under) sample the minority (majority) class.
3. Syntehise the feautures of the minory class to manufacture plausible new instances of the minority class (the SMOTE technique)
3. Choose an algorithm that can mitigate the imbalanced class representation
4. Gather more data (not feasible if the imbalance is intrinsic to the process as it is the case in churn/anomaly/fraud detection, etc.)

More information:
- https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/
- SMOTE: https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

For the step-by-step development I'd proceed as follows:
1. Ignore the "elephant in the room" to firstly focus on the overall workflow
2. Use the confusion matrix (or similar) as a peformance measure
3. Pick an easy solution to tackle the imbalanced classification (e.g. over-sample the minority class)

### Remove obviously-not-needed columns
The dataset author tells that he's been working on Naive Bayes classifier with no satisfactory outcome.
Therefore the corresponding columns are rather *outputs* than *inputs* (features) and let's drop them together with the client number.

In [7]:
for col in [
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2",
    "CLIENTNUM",
    ]:
    print(col)
    df = df.drop(col, axis=1)

Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1
Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
CLIENTNUM


## Translate text to numbers 

### How to transform the text values?
- "Income_Category" is actually a numerical column, except it needs a transformation (e.g. assigning mean of the range)
- "Education_Level" and "Card_Category" represent some progression, except the former is unsorted
  - We'll sort the former by a dedicated function and assign a numerical value
  - We'll use an ordinal encoder for the latter
- "Marital_Status" doesn't exhibit a progression, but rather "one of many types" pattern; it may be tackled with one-hot encoder
- "Gender" would also suit to one-hot encoder (or perhaps an ordinal encoder, since there are 2 values only?)


### How to deal with unknown values?
- The (tranformed into) numerical values may be assigned the mean
- The 'unknown' marital status or gender is more challenging; we can e.g.
  - skip these features altogether, not to bias the algorithm
  - assign a random value according to the feature distribution
  - assign their most frequent values
  - skip the instances (i.e. customers) who didn't provide either of these
- To simplify let's skip the marital status and gender altogether


In [8]:
def sort_edu(val):
    "Sort education level correctly"
    order = ["Uneducated", "High School", "College", "Graduate", "Post-Graduate", "Doctorate"]
    if val in order:
        return order.index(val)
    # For unknowns return NaN to later on apply median
    return np.nan

#df['Education'] = df.apply(lambda x: sort_edu(x["Education_Level"]), axis=1)
df['Education_Level'] = df['Education_Level'].map(sort_edu)

In [9]:
def avg_income(val):
    'Replace income ranges with range averages using regexps, eg. "$60K - $80K" -> 70'
    if val is np.nan:
        return np.nan
    if val == 'Unknown':
        return np.nan
    import re
    matches = re.findall("\$[0-9]*K", val)
    matches = [float(x.replace('$','').replace('K','')) for x in matches]
    if len(matches) == 1:
        return matches[0]
    return sum(matches) / len(matches)
#avg_income("$60K - $80K")
#df['Avg_Income'] = df.apply(lambda x: avg_income(x['Income_Category']), axis=1)
df['Income_Category'] = df['Income_Category'].map(avg_income)
df['Income_Category']

0         70.0
1         40.0
2        100.0
3         40.0
4         70.0
         ...  
10122     50.0
10123     50.0
10124     40.0
10125     50.0
10126     40.0
Name: Income_Category, Length: 10127, dtype: float64

### Build transformation pipeline: encode text-based values and standardisation

In [10]:
# Load sklearn transformation tools
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Numerical pipeline
num_cols_names = df.select_dtypes(exclude=np.object).columns
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])

# Ordinal categories' pipeline
ordinal_cols_names = ["Card_Category"]

# Full pipeline
# ColumnTransformer executes the pipelines on specified lists of column names
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_cols_names),
    ('ordered', OrdinalEncoder(), ordinal_cols_names)
    #('cat', OneHotEncoder(), text_cols_names),
])

# Split feature matrix and the label vector
X = df.drop("Attrition_Flag", axis=1)
y = df['Attrition_Flag'].copy()

# Transform the feauture matrix with the pipeline
X_tr = full_pipeline.fit_transform(X)

# Transform the label vector using the LabelEncoder
le = LabelEncoder()
y_tr = le.fit_transform(y)
print("Percentage of positive class:", y_tr.sum()/y_tr.shape[0])
print("Meaning of 0-class and 1-class:", le.inverse_transform([0,1]))
print("Num cols:", num_cols_names)
print("Ordinal cols:", ordinal_cols_names)

Percentage of positive class: 0.8393403772094401
Meaning of 0-class and 1-class: ['Attrited Customer' 'Existing Customer']
Num cols: Index(['Customer_Age', 'Dependent_count', 'Education_Level', 'Income_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio'],
      dtype='object')
Ordinal cols: ['Card_Category']


For the next paragraphs, the notions of true/false postive/negative will be used.
The `LabelEncoder` classified active clients as _positive_ and the churned ones as _negatives_.
Actually, it would be more convenient to define them the other way round: a churning client should be the positive outcome of the model (similar to detecting any rare signal, e.g. disease, fraud, etc.).

In [11]:
y_tr = np.where(y_tr==0, 1, 0) # swap 0 and 1
y_tr.sum() / y_tr.shape[0]

0.1606596227905599

## Create train and test sets
Despite we're only at the cleaning stage, it's a good practice to already seperate out the test set from the training one to avoid (analyst's) biases.

Let's use 20% of the dataset as the test sample.
To ensure representative sampling we're using the stratified shuffle split: the feautures distributions should agree between the train and test samples.
However, stratified sampling does not address the above-mentioned _imbalance_ between the class labels.
TODO: Let's keep that in mind and tackle at a later stage.

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit
# >>> for train_index, test_index in sss.split(X, y):
# ...     print("TRAIN:", train_index, "TEST:", test_index)
# ...     X_train, X_test = X[train_index], X[test_index]
# ...     y_train, y_test = y[train_index], y[test_index]
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X_tr, y_tr):
    X_train, X_test = X_tr[train_index], X_tr[test_index]
    y_train, y_test = y_tr[train_index], y_tr[test_index]
    
print(y_train.sum()/y_train.shape[0])
print(y_test.sum()/y_test.shape[0])

0.1607208986544871
0.16041461006910168


### Saving the dataset for the next step
For the sake of clarity, let's save the current dataset using the `pickle` module, before further steps.
In the next notebook we'll explore the data and engineer the features.

In [13]:
X_train.shape
#len(num_cols_names)
list(list(num_cols_names)+list(ordinal_cols_names))


['Customer_Age',
 'Dependent_count',
 'Education_Level',
 'Income_Category',
 'Months_on_book',
 'Total_Relationship_Count',
 'Months_Inactive_12_mon',
 'Contacts_Count_12_mon',
 'Credit_Limit',
 'Total_Revolving_Bal',
 'Avg_Open_To_Buy',
 'Total_Amt_Chng_Q4_Q1',
 'Total_Trans_Amt',
 'Total_Trans_Ct',
 'Total_Ct_Chng_Q4_Q1',
 'Avg_Utilization_Ratio',
 'Card_Category']

In [14]:
# Save as Pandas DataFrame
df_tr = pd.DataFrame(X_train, columns=list(num_cols_names)+ordinal_cols_names)
df_tr.head()
# Append the labels column
df_tr["Label"] = y_train
# Write file
df_tr.to_pickle("Xy_train_df_step1.pickle")
df_tr.head()

,Customer_Age,Dependent_count,Education_Level,Income_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Card_Category,Label
0,-1.288101,-1.806378,-0.039873,-0.837746,-1.493661,0.120579,0.651940,-0.411616,-0.667004,1.309516,-0.784263,-0.337327,-0.779840,-0.973895,-1.109831,2.151448,0.0,0
1,-0.290150,0.503368,-0.807534,0.312139,0.008965,0.763943,-0.337598,0.492404,1.831365,-1.426858,1.958900,0.406300,-0.624701,-1.144315,-0.542782,-0.997155,3.0,1
2,-0.040662,-0.266547,-0.807534,1.462024,-0.742348,1.407306,0.651940,-1.315636,0.334280,-0.304086,0.361471,0.602472,-0.037412,1.028541,0.595518,-0.714216,0.0,0
3,-1.288101,-0.266547,0.727789,-0.837746,-1.493661,-0.522785,-1.327136,0.492404,-0.606047,0.527870,-0.653244,0.497543,-0.810161,-1.016500,-1.450061,0.852830,0.0,0
4,0.458314,-1.036462,0.727789,2.228614,0.509840,0.120579,-0.337598,0.492404,2.848054,0.027224,2.845015,-0.159404,-0.156047,0.304255,0.007467,-0.873823,0.0,0


In [15]:
# Write features to file
import pickle
with open('features.pickle', 'wb') as f:
    pickle.dump(list(num_cols_names)+ordinal_cols_names, f)

In [16]:
np.save("X_train_step1.pickle", X_train)
np.save("X_test_step1.pickle", X_test)
np.save("y_train_step1.pickle", y_train)
np.save("y_test_step1.pickle", y_test)